In [7]:
import re
import tor

text = "回复@魏小河才不是乖乖的小盆宇:完全控制以前都不能放松"
at_pattern = re.compile("@\S*:", re.S)
text = re.sub(at_pattern, "", text)
at_pattern = re.compile("@\S*:", re.S)
text = re.sub(at_pattern, "", text)
print(text.strip())
print(torch.cuda.is_available())

回复完全控制以前都不能放松
False
